In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
from sqlalchemy import create_engine
from config import config

In [2]:
# Obtain the configuration parameters
conn = psycopg2.connect(**config())
# Create a new cursor
cur = conn.cursor()

In [ ]:
# get main German cities codes

cities_dict = {
    'Dusseldorf': 100207,
    'Berlin': 87372,
    'Essen': 102157,
    'Munchen': 121673,
    'Koln': 113144,
    'Stuttgart': 143262,
    'Dresden': 100051,
    'Hannover': 109489,
    'Dortmund': 99990,
    'Frankfurt am Main': 105043,
    'Hamburg': 109447
}
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
# get all offers quantity - haus und wohnung

cities_offers = []

for city, code in cities_dict.items():
    
    total_offers = 0
    
    for i in range(1,3):
        url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city={code}&marketingtype=2&page=1'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Find number of rent offers.
        a = soup.findAll('h1', class_='box-50')[0].get_text()
        total_offers += int(re.search('\d+', a).group())
    
    cities_offers.append({'extraction_datetime': now, 'city': city, 'city_code': code, 'offers': total_offers})
    
offers_by_page = len(soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter"))    
df_offers = pd.DataFrame(cities_offers)
df_offers.to_csv(f'../data/total_offers_by_city_{now2}.csv', index=False)

### Pipeline test

In [48]:
import logging

In [49]:
import os
import re
import math
import requests
import psycopg2
import logging
import numpy as np
import pandas as pd
import psycopg2.extras as extras

from time import sleep
from random import randint
from bs4 import BeautifulSoup
from datetime import datetime
from sqlalchemy import create_engine
from config import config


def connect(conf):
    """ 
    Connect to the PostgreSQL database server 
    
    Parameters:
    ----------
        conf: configuration for connection
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        logging.info('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**conf)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    logging.info("Connection successful")
    
    return conn

def get_offers_qtt():
    '''
    Get offers quantity by city and store it in a postgres DB.
    '''
    
    cities_dict = {
        'Dusseldorf': 100207,
        'Berlin': 87372,
        'Essen': 102157,
        'Munchen': 121673,
        'Koln': 113144,
        'Stuttgart': 143262,
        'Dresden': 100051,
        'Hannover': 109489,
        'Dortmund': 99990,
        'Frankfurt am Main': 105043,
        'Hamburg': 109447
    }
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    # get all offers quantity - haus und wohnung

    cities_offers = []

    for city, code in cities_dict.items():
        
        logging.info(f'Getting offers in {city}...')

        total_offers = 0

        for i in range(1,3):
            url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city={code}&marketingtype=2&page=1'
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, "html.parser")

            # Find number of rent offers.
            a = soup.findAll('h1', class_='box-50')[0].get_text()
            total_offers += int(re.search('\d+', a).group())

        cities_offers.append({'extraction_datetime': now, 'city': city, 'city_code': code, 'offers': total_offers})

    # offers_by_page = len(soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter"))    
    df_offers = pd.DataFrame(cities_offers)
    #df_offers.to_csv(f'temp_data/total_offers_by_city_temp_file.csv', index=False)
    
    return df_offers


def load_results(conn, df, table_name):
    """Get the extracted data and insert into a postgres table.
    
    Parameters:
    -----------
        conn: connection to the database.
        df: Dataframe to insert into the database.
        table_name: table to insert the dataframe. 
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table_name, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        conn.rollback()
        cursor.close()
        return 1
    print(f"{table_name} uptodate.")
    logging.info(f"{table_name} uptodate.")
    cursor.close()
    
    
conn = connect(conf=config())

df = get_offers_qtt()

load_results(conn, df, table_name='offers_qtt_by_city')

In [8]:
a = [i for i in range(0, 20000, 500)]

In [9]:
a

[0,
 500,
 1000,
 1500,
 2000,
 2500,
 3000,
 3500,
 4000,
 4500,
 5000,
 5500,
 6000,
 6500,
 7000,
 7500,
 8000,
 8500,
 9000,
 9500,
 10000,
 10500,
 11000,
 11500,
 12000,
 12500,
 13000,
 13500,
 14000,
 14500,
 15000,
 15500,
 16000,
 16500,
 17000,
 17500,
 18000,
 18500,
 19000,
 19500]